[Parent Notebook from DASMEHDIXTR](https://www.kaggle.com/code/dasmehdixtr/face-age-classification-will-be-updated)

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
import os

from matplotlib import pyplot as plt

from sklearn import metrics

In [ ]:
# Load training data and print the first three rows
data = pd.read_csv('/kaggle/input/faces-age-detection-dataset/train.csv')
data.head(3)

In [ ]:
# Print all categories in the class variables
data['Class'].unique()

In [ ]:
# Create a histogram for all categories in the class variable.
data['Class'].hist()

In [ ]:
# One-hot encoding for changing string values to numeric values. 
data['Class'].replace(['YOUNG', 'MIDDLE','OLD'],
                        [0, 1, 2], inplace=True)
data.head(3)

In [ ]:
# Methods for loading images.
def imageReader(path,ch = 3, resize=(512,512)):
	kp = tf.io.read_file(path)
	kp = tf.image.decode_jpeg(kp, channels=ch)
	kp = tf.image.convert_image_dtype(kp, dtype=tf.float32)
	kp = tf.image.resize(kp, resize)
	return kp

def load_data(image_path, label):
    image = imageReader(image_path, 3, (512,512))
    
    return (image, label)

In [ ]:
# Initializing image paths variables.
image_paths = glob.glob('/kaggle/input/faces-age-detection-dataset/Train/*.jpg')
image_paths = image_paths[0:200]
print(len(image_paths))

In [ ]:
# Initialize label values
label_list = []
for i in image_paths:
    _,tail = os.path.split(i)
    label = data.loc[data['ID'] == tail]['Class'].values[0]
    label_list.append(label)
# print(len(label_list))

In [ ]:
# Create Train, Test, and Validation datasets
train_size = int(0.8*(len(image_paths)))
test_size = 20

trainDS = tf.data.Dataset.from_tensor_slices((image_paths[:train_size], label_list[:train_size]))
testDS = tf.data.Dataset.from_tensor_slices((image_paths[train_size:train_size + test_size], label_list[train_size :train_size + test_size]))
validationDS = tf.data.Dataset.from_tensor_slices((image_paths[train_size + test_size:], label_list[train_size + test_size:]))

In [ ]:
# Creating Autotune objects
# For Autotune objects the prefetch buffer sizes are automitically tuned. 
AUTOTUNE = tf.data.AUTOTUNE
trainDS = (trainDS
    .map(load_data, num_parallel_calls=AUTOTUNE)
    .batch(64)
    .prefetch(AUTOTUNE)
)
print(trainDS)

In [ ]:
# Adding data agumentation to training images.
# Data agumentation adds variation to training data
trainDS = trainDS.map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).cache(
).map(
    lambda image, label: (tf.image.per_image_standardization(image), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.4, upper=0.6), label)
).map(
    lambda image, label: (tf.image.random_brightness(image, max_delta = 0.4), label)
).map(
    lambda image, label: (tf.image.random_hue(image, max_delta = 0.4), label)
).map(
    lambda image, label: (tf.image.random_saturation(image, lower=0.4, upper=0.6), label)
).shuffle(
    1000
).repeat(2)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
validationDS = (validationDS
    .map(load_data, num_parallel_calls=AUTOTUNE)
    .batch(64)
    .prefetch(AUTOTUNE)
)

In [ ]:
# Adding data agumentation to validation images.
validationDS = validationDS.map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).cache(
).map(
    lambda image, label: (tf.image.per_image_standardization(image), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.4, upper=0.6), label)
).map(
    lambda image, label: (tf.image.random_brightness(image, max_delta = 0.4), label)
).map(
    lambda image, label: (tf.image.random_hue(image, max_delta = 0.4), label)
).map(
    lambda image, label: (tf.image.random_saturation(image, lower=0.4, upper=0.6), label)
).shuffle(
    1000
).repeat(2)

In [ ]:
testDS = (testDS
    .map(load_data, num_parallel_calls=AUTOTUNE)
    .batch(64)
    .prefetch(AUTOTUNE)
)

In [ ]:
# Standardizing the test data.
testDS = testDS.map(
    lambda image, label: (tf.image.per_image_standardization(image), label)
)

In [ ]:
# Using pre-trained models for creating model
# Pre-training models are usually used for images because training models for interpreting 
# images needs a lot of computer resources and time. 
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
model = tf.keras.applications.MobileNetV3Large(input_shape = (512,512,3,),include_top=False,weights='imagenet')
# model.summary()

In [ ]:
# Set model.trainable to false, this prevents changing weights of pre-trained model.
model.trainable = False

# Adding custom layers to the pre-trained model.
inputs = tf.keras.Input(shape=(512, 512, 3))
x = preprocess_input(inputs)
x = model(x, training=True)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1000)(x)
x = tf.keras.layers.Dense(600)(x)
x = tf.keras.layers.Dense(600)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(400)(x)
x = tf.keras.layers.Dense(400)(x)
x = tf.keras.layers.Dense(100)(x)
x = tf.keras.layers.Dense(100)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(50)(x)
x = tf.keras.layers.Dense(50)(x)
x = tf.keras.layers.Dense(20)(x)
x = tf.keras.layers.Dense(20)(x)
outputs = tf.keras.layers.Dense(3)(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
# Training the model
history = model.fit(trainDS, epochs=17, validation_data=validationDS)

In [ ]:
# Plotting the train results.
plt.plot(history.history['sparse_categorical_crossentropy'])
plt.plot(history.history['loss'])
plt.title('model sparse_categorical_crossentropy')
plt.ylabel('sparse_categorical_crossentropy')
plt.xlabel('epoch')
plt.legend(['sparse_categorical_crossentropy', 'Loss'], loc='upper left')
plt.show()

In [ ]:
# Predict probabilites of test data.
probabilities = model.predict(testDS)
# print(probabilities)
# Create classes from predictions
predictions = np.argmax(probabilities,axis=1)
actual_values = label_list[train_size :train_size + test_size]
print(actual_values)
print(predictions)

In [ ]:
print("balanced accuracy:   %0.3f" % metrics.balanced_accuracy_score(actual_values, predictions))
print("accuracy:   %0.3f" % metrics.accuracy_score(actual_values, predictions))